In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import requests
from bs4 import BeautifulSoup
import string 
import spacy 
import re

In [4]:
import urllib.request

In [5]:
dflinks = pd.read_excel('test.xlsx')

In [6]:
dflinks

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
107,146,https://insights.blackcoffer.com/blockchain-fo...
108,147,https://insights.blackcoffer.com/the-future-of...
109,148,https://insights.blackcoffer.com/big-data-anal...
110,149,https://insights.blackcoffer.com/business-anal...


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [8]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [9]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [WinError 10060] A connection attempt failed because
[nltk_data]     the connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>


False

In [10]:
from nltk.corpus import stopwords

In [11]:
from nltk.tokenize import word_tokenize

In [12]:
import textstat

In [13]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0",
}


In [21]:
# Helper Functions

def vovel_counter(word):
    word = word.lower()
    count = 0
    vovels = 'aeiouy'
    if word[0] in vovels:
        count += 1
    for index in range(len(word)):
        if word[index] in vovels and word[index - 1] not in vovels:
            count += 1
            if word.endswith('e'):
                count -= 1
    if count == 0:
        count += 1
    return count

from textblob import TextBlob
    # Get The Subjectivity with creating a function for the same to get the sentiment and subjectivity 
def sentiment_analysis(data):
    sentiment = TextBlob(str(data["content"])).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])

def propernoun(text):
    count = 0
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        works = nltk.word_tokenize(sentence)
        tagged = nltk.pos_tag(works)
        for (word, tag) in tagged:
            if tag == "PRP":
                count += 1
    return count

In [20]:
str(data["content"])

'0    []\nName: content, dtype: object'

In [26]:
df = pd.DataFrame(columns=['url_id', 'url','content', 'Posiitive_score', 'Negative_Score', 'AVG SENTENCE LENGTH', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH', "polarity", "subjectivity"])

In [ ]:
df

,url_id,url,content,Posiitive_score,Negative_Score,AVG SENTENCE LENGTH,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,polarity,subjectivity


# test1

In [41]:
for link in dflinks.iterrows():
    url = link[1]['URL']
    urlid = link[1]['URL_ID']
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.content, 'html.parser')

    try:
        title = soup.find("h1", class_="entry-title")
    except AttributeError:
        pass
    
    
    try:
        title = title.text.replace("\n", " ")
    except AttributeError:
        pass
    
    
    print(title)


    content=soup.findAll(attrs={'class':'td-post-content'})
    
    try:
        content=content[0].text.replace('\n'," ")
    except IndexError:
        pass

    
    try:
        content = content.translate(str.maketrans(" ", " ", string.punctuation))
    except AttributeError:
        pass

    
    try:
        txttoken = word_tokenize(content)
    except TypeError:
        txttoken = word_tokenize(str(content))


    print(' ')
    print(' ')
    print('text_token : ', len(txttoken))


    corpus = stopwords.words('english')
    corpus.append('the')
    removtokenitems = [word for word in txttoken if not word in corpus] # Removes stop waords from the main word corpus

    with open('MasterDictionary\positive-words.txt', "r") as pos:      # opening the possitive text file and assigning variables 
        posw = pos.read().split('\n')                 # Reading the file while removing the endcoding function 
        posw = posw[:50]

    pos_count = " ".join([w for w in removtokenitems if not w in posw]) # Adsd a space to each word what can be found in the possitive list of woard 
    pos_count = pos_count.split(" ") # After cleaning the possitive words add a " " space between words to get them counted


    Posiitive_score = len(pos_count) # Counting the number of words in the short listed corpus
    print(r"Positive Score = {}".format(Posiitive_score))

    with open("MasterDictionary/negative-words.txt", 'r') as neg:
        negw = neg.read().split('\n')
        negw = negw[5:]

    neg_count = " ".join([w for w in removtokenitems if not w in negw])
    neg_count = neg_count.split(" ")

    Negative_Score = len(neg_count)
    print(r"Negative Score = {}".format(Negative_Score))

    filtcontent = "".join(removtokenitems)
    try:
        ASL = len(content.replace(' ', ''))/len(re.split(r'[?!.]', content)) # removing spaces and dividing the content with "!, ?, ."
    except AttributeError:
        len(str(content).replace(' ', ''))/len(re.split(r'[?!.]', str(content)))

    try:
        FOG_INDEX=(textstat.gunning_fog(content)) #  Usig Gunnig fox library for FOX index 
    except TypeError:
        FOG_INDEX=(textstat.gunning_fog(str(content)))
    try:
        WPS = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()] # With the same principle striping the sentence to count the words 
    except TypeError:
        WPS = [len(l.split()) for l in re.split(r'[?!.]', str(content)) if l.strip()] # With the same principle striping the sentence to count the words 


    WPS = sum(WPS)/len(WPS)

    try:
        complex_words = vovel_counter(content)
        word_count = len(content)
        personal_pronoun = propernoun(content)
        Avg_word_count = len(content.replace(" ", ""))/len(content.split())
    except AttributeError:
        complex_words = vovel_counter(str(content))
        word_count = len(str(content))
        personal_pronoun = propernoun(str(content))
        Avg_word_count = len(str(content).replace(" ", ""))/len(str(content).split())


    try:
        word = content.replace(" ", "")
    except AttributeError:
        word = str(content).replace(" ", "")
        
    vovelcount = 0
    for w in word: # count vovel if any of the vovel words were to be found in the content and add to the vovel count 
        if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            vovelcount +=1

    try:
        vovelperword = vovelcount / len(content.split())
    except AttributeError:
        vovelperword = vovelcount / len(str(content).split())


    data = [[str(urlid), url, content, Posiitive_score, Negative_Score, ASL, FOG_INDEX, WPS, complex_words, word_count, vovelperword, personal_pronoun, Avg_word_count]]  # Creating the list of items to be added to each columns and form a dataframe
    data = pd.DataFrame(data, columns=['url_id', 'url','content', 'Posiitive_score', 'Negative_Score', 'AVG SENTENCE LENGTH', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS',	'AVG WORD LENGTH']) # Adding column names in the dataframe

    print('')

    # Adding Subjectivity & Polarity
    data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1) # Dumping the values of polarity and subjectivity to the values 

    df = df.append(data, ignore_index=True)
    


AI in healthcare to Improve Patient Outcomes
 
 
text_token :  1814
Positive Score = 1193
Negative Score = 1166



C:\Users\Pulkit Gaur\AppData\Local\Temp\ipykernel_14124\3521568910.py:126: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


What if the Creation is Taking Over the Creator?
 
 
text_token :  1450
Positive Score = 818
Negative Score = 783



C:\Users\Pulkit Gaur\AppData\Local\Temp\ipykernel_14124\3521568910.py:126: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


What Jobs Will Robots Take From Humans in The Future?
 
 
text_token :  1723
Positive Score = 1059
Negative Score = 1028



...

C:\Users\Pulkit Gaur\AppData\Local\Temp\ipykernel_14124\3521568910.py:126: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


Business Analytics In The Healthcare Industry
 
 
text_token :  723
Positive Score = 447
Negative Score = 444



C:\Users\Pulkit Gaur\AppData\Local\Temp\ipykernel_14124\3521568910.py:126: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


Challenges and Opportunities of Big Data in Healthcare
 
 
text_token :  1046
Positive Score = 629
Negative Score = 591



C:\Users\Pulkit Gaur\AppData\Local\Temp\ipykernel_14124\3521568910.py:126: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


In [42]:
data

,url_id,url,content,Posiitive_score,Negative_Score,AVG SENTENCE LENGTH,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,polarity,subjectivity
0,150,https://insights.blackcoffer.com/challenges-an...,Big Data To begin with I shall first like to ...,629,591,5271.0,422.21,1045.0,1942,6319,2.03445,22,5.044019,0.124963,0.45902


In [47]:
df = df[['url_id', 'url','content', 'Posiitive_score', 'Negative_Score', 'polarity', 'subjectivity', 'AVG SENTENCE LENGTH', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS',	'AVG WORD LENGTH']]

In [48]:
df

,url_id,url,content,Posiitive_score,Negative_Score,polarity,subjectivity,AVG SENTENCE LENGTH,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1193,1166,0.136936,0.463364,10162.0,721.92,1790.0,3579,11955,2.248045,22,5.677095
1,38,https://insights.blackcoffer.com/what-if-the-c...,Human minds a fascination in itself carrying ...,818,783,0.074693,0.433596,6861.0,567.40,1409.0,2424,8275,1.948900,51,4.869411
2,39,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1059,1028,0.111470,0.483201,9220.0,684.87,1700.0,3340,10922,2.187647,28,5.423529
3,40,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarterthan...,989,965,0.141419,0.487387,8061.0,660.88,1650.0,2877,9715,1.951515,49,4.885455
4,41,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1067,1042,0.053178,0.509139,8927.0,699.95,1739.0,3134,10671,2.015526,53,5.133410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,146,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,582,557,0.035488,0.402814,4895.0,369.76,911.0,1697,5807,2.109769,23,5.373216
108,147,https://insights.blackcoffer.com/the-future-of...,What Is an Investment An investment is a reso...,1014,1002,0.074727,0.389435,7979.0,620.72,1550.0,2777,9546,2.042581,24,5.147742
109,148,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,727,685,0.040319,0.417892,5903.0,465.90,1155.0,2147,7071,2.069264,14,5.110823
110,149,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,447,444,0.208748,0.546448,4105.0,291.93,715.0,1461,4816,2.281119,9,5.741259


In [49]:
df.to_excel('Final_output.xlsx', index=False)